In [767]:
import random
from random import randint
import pandas as pds

In [768]:
names = ["John", "Harry", "Mary", "Alex","Pamela", "Robin","Francis","Charles"]
measurement_height = ["inches", "cm", "meters", "feet"]
items_for_rent_list = ["cars","books","computers","buses","houses","bikes"]
days_in_a_week = ["Sunday","Monday","Wednesday","Thursday","Friday","Saturday"]
measurement_liquid = ["ml", "millileters","litres", "L", "Lit","ounces","oz","gallons"]
containers_list = ["bottles","jars","tanks","tumblers","mugs","bowls"]  
measurement_weight = ["kgs","kilograms","kilos","pounds","mg","milligrams","grams","gm"]
vegetables = ["tomatoes","onions","capsicum","mushrooms","carrots"]
store_type = ["grocery store", "super market","salon", "shop","field","school"]
duration_list = ["days","weeks","years","months"]
activity_list = ["walk","run","jog","race","speed walk","brisk walk"]
measurement_distance = ["km","mts","cm","mm","miles"]
options_set = ["Both A and B","Both a and b","Both a and c","Both C and A", "Both Option 1 and Option 2","First letter of apple and first letter of banana","Both B and C","Options C and B","First letter of charlie and third letter of subway","All the above","None of the above"]
month_list = ["January","February","March","April","May","June","July","August","September","October","November","December"]

In [769]:
json_dataset = []
duplicity = 100

In [770]:
Options_set_0 = ["None of the above","Given options do not satisy","No valid option"] 
Options_set_1 = ["Options a and b","Options a, b and c","All the above","Options b and c","Option d","Options a and c","None of the above","Given options do not satisy","No valid option"]
Options_set_2_ab = ["Both a and b","Both b and a"] 
Options_set_2_bc = ["Both b and c","Both c and b"]
Options_set_2_ac = ["Both a and c","Both c and a"]
Options_set_3 = ["All the above","Options a, b and c"]

In [771]:
def replace_parameters(template,parameters):
    for i in parameters:
        k = template.find("$",0,len(template))
        if k!=-1:
            template = template[:k]+i+template[k+2:]
    return template

In [772]:
#******************************************************************************************************#
#  Template 1 : Based on the Capacity of different vehicles.  Creating different set of options       *#
#******************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Vehicle': str},sheet_name='template_1')  
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Category': str},sheet_name='human_category') 
df = pd.DataFrame(data,columns=['Vehicle'])   # Ordered vehicles by capacity in increasing order
df_1 = pd.DataFrame(data_1,columns=['Category']) 

template_1 = "If $A need $1 $B each having maximum capacity of $2 to travel, then which of the following options could be the number of $C having maximum capacity of $3 that the $A can use for the travel?"

Vehicle_list = df['Vehicle'].tolist()
Category_list = df_1['Category'].tolist()
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    A = random.choice(Category_list)
    param_1 = randint(2,100)
    B = random.choice(Vehicle_list)
    param_2 = randint(4,100)
    C_list = []
    C_list = Vehicle_list.copy()
    C_list.remove(B)
    C = random.choice(C_list)
    index_1 = Vehicle_list.index(B)
    index_2 = Vehicle_list.index(C)
    if index_1 < index_2 : #Second vehicle should ideally have more capacity than first
        param_3 = randint(param_2+1,1000)
    else:                  #Second vehicle capacity should be less than first
        param_3 = randint(2,param_2-1)
        
    parameters = [A,str(param_1),B,str(param_2),C,str(param_3),A]
  
    #questions
    QA["question"] = replace_parameters(template_1,parameters)    
    print(QA["question"])
    
    # answers 
    total_no_of_people = param_1*param_2
    # maximum number of vehicles of type 2 required = total no of people (assuming each person can take one vehicle)
    max_1 = total_no_of_people
    #calculate the minimum number of vehicles of type 2 required
    if total_no_of_people % param_3 == 0:
        min_1 = int(total_no_of_people / param_3) 
        option_invalid = randint(0,min_1-1)
    else:
        min_1 = int(total_no_of_people / param_3)+1
        option_invalid = randint(0,min_1)  

    
    # Generate Answers and Options for any of the following sets
    #------------------------------------------------------------
    # To randomly choose options set for each question
    # 0 - Options with no correct answer
    # 1 - Options with one correct answer 
    # 2 - Options with two correct answers
    # 3 - Options with three correct answers
    
    if max_1-min_1>3: #checking if atleast three answers are possible
        ans_1 = randint(min_1+1,max_1) 
        ans_2 = ans_1
        while ans_1 == ans_2:
            ans_2 = randint(min_1+1,max_1)
        ans_3 = ans_2
        while ans_3 == ans_1 or ans_3 == ans_2:
            ans_3 = randint(min_1+1,max_1)
    else:
        continue
        i = i-1
    
    
    # generate options set with incorrect answers 
    options_set_no_ans = [max_1+randint(2,1000), max_1+1,0, int(-1 *randint(2,1000)),option_invalid] 
    if param_2 != param_3:
        options_set_no_ans.append(param_1)
    if param_2 > param_3:  #if capacity of vehicle 1 is > capacity of vehicle 2
        options_set_no_ans.append(randint(1,param_1-1))  #answer cannot be less than param_1
    
    
    choose_options_set = randint(0,3)  
    if choose_options_set == 0: # no correct answers
        set_1 = []
        set_1 = options_set_no_ans.copy() 
        option_1 = random.choice(set_1)
        set_1.remove(option_1)
        option_2 = random.choice(set_1)
        set_1.remove(option_2)
        option_3 = random.choice(set_1)
        options_list = [option_1,option_2,option_3]
        option_4 = random.choice(Options_set_0)
        options_list.append(option_4)
        ans_t1 = option_4
        
    elif choose_options_set == 1:  # one correct answer 
        options_list = [ans_1,option_invalid,random.choice(options_set_no_ans)]    
        random.shuffle(options_list)
        options_list.append(random.choice(Options_set_1))
        ans_t1 = ans_1
        
    elif choose_options_set == 2: # two correct answers   
        options_list = [ans_1,ans_2,random.choice(options_set_no_ans)]
        random.shuffle(options_list)
        if (options_list[0] == ans_1 or options_list[0] == ans_2) and (options_list[1] == ans_1 or options_list[1] == ans_2):
            option_4 = random.choice(Options_set_2_ab)
        elif (options_list[1] == ans_1 or options_list[1] == ans_2) and (options_list[2] == ans_1 or options_list[2] == ans_2):
            option_4 = random.choice(Options_set_2_bc)         
        else:
            option_4 = random.choice(Options_set_2_ac)
        options_list.append(option_4)
        ans_t1 = option_4
    else:            # all correct answers
        options_list = [ans_1,ans_2,ans_3]
        random.shuffle(options_list)
        option_4 = random.choice(Options_set_3)
        options_list.append(option_4)
        ans_t1 = option_4        
                 
    QA["options"] = options_list
    print(QA["options"])
    QA["answer"] = ans_t1
    print(QA["answer"])

    json_dataset.append(QA)
    
    i=i+1;


If crew members need 22 Jet planes each having maximum capacity of 94 to travel, then which of the following options could be the number of Motor bikes having maximum capacity of 82 that the crew members can use for the travel?
[107, 1928, 195, 'Options a, b and c']
Options a, b and c
If sales women need 86 Trains each having maximum capacity of 49 to travel, then which of the following options could be the number of Hover crafts having maximum capacity of 20 that the sales women can use for the travel?
[80, -191, 4215, 'Given options do not satisy']
Given options do not satisy
If painters need 80 Trolley buses each having maximum capacity of 95 to travel, then which of the following options could be the number of Jet scooters having maximum capacity of 30 that the painters can use for the travel?
[1140, 80, 214, 'None of the above']
1140
If colleagues need 39 Jet scooters each having maximum capacity of 93 to travel, then which of the following options could be the number of Kayak boa

In [773]:
#***********************************************************************************************************#
#  Template 2 : Based on comparision keywords (more than, less than) and rankings (1st, 2nd, 3rd etc...)   *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])   
names = df['Names'].tolist()  

keywords_t2 = ["less","more"]
template_2 = "In a list of $1 shortest people in the world, if $A's height is $B than $x times of the $a$q shortest person whose height is $2 inches and $C than $y times of $b$r shortest person whose height is $3 inches. Which of the following can be the height of $A?"

def find_2(x1):
    if x1==1:
        res="st"
    elif x1==2:
        res = "nd"
    elif x1==3:
        res = "rd"
    else:
        res="th"
    return res      
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    param_1 = randint(10,100)
    A = random.choice(names)
    B = random.choice(keywords_t2)
    if B=="more":
        C = "less"
        x = randint(2,20) # y>x , so param_3 > param_2
        y = randint(x+1,40)
        a = randint(1,param_1-1)
        b = randint(a+1,param_1)
        param_2 = randint(5,20)
        param_3 = randint(param_2+1,22)   
        
    else:
        C="more"
        y = randint(2,20) # x> y, so param_2 > param_3
        x = randint(y+1,40)
        b = randint(1,param_1-1)
        a = randint(b+1,param_1) 
        param_3 = randint(5,20)
        param_2 = randint(param_3+1,22) 
    
    q = find_2(a)
    r = find_2(b)
    parameters = [str(param_1),str(A),str(B),str(x),str(a),str(q),str(param_2),str(C),str(y),str(b),str(r),str(param_3),str(A)]
    
    #questions
    QA["question"] = replace_parameters(template_2,parameters)    
    print(QA["question"])

    # answers
    if B=="more":
        ans_t2 = randint(param_2*x+1,param_3*y-1)
        other_options = [param_3, param_2,param_2*x,param_3*y,0]
    else:
        ans_t2 = randint(param_3*y+1,param_2*x-1)
        other_options = [param_1, param_2,param_2*x,param_3*y,0]
    
    #options
    Option_2 = random.choice(other_options)
    Option_3 = random.choice(other_options)
    options_list = [ans_t2,Option_2,Option_3]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = height_ans
    
    
    json_dataset.append(QA)

    i+=1
    


In a list of 56 shortest people in the world, if Zahir's height is more than 11 times of the 43th shortest person whose height is 19 inches and less than 35 times of 45th shortest person whose height is 22 inches. Which of the following can be the height of Zahir?
In a list of 17 shortest people in the world, if Caden's height is less than 37 times of the 14th shortest person whose height is 20 inches and more than 4 times of 5th shortest person whose height is 15 inches. Which of the following can be the height of Caden?
In a list of 56 shortest people in the world, if Axel's height is more than 3 times of the 17th shortest person whose height is 18 inches and less than 24 times of 32th shortest person whose height is 19 inches. Which of the following can be the height of Axel?
In a list of 57 shortest people in the world, if Houston's height is less than 34 times of the 46th shortest person whose height is 12 inches and more than 15 times of 36th shortest person whose height is 5 inc

In [774]:
#***********************************************************************************************************#
#  Template 3 : Based on the occurence of events                                                           *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Event':str,'Year': int},sheet_name='template_3')  
df = pd.DataFrame(data,columns=['Event','Year'])
print(df)

template_3 = "$A took place in the year $1. When could be the year in which $B took place?"

i=0
list_A = df['Event'].tolist()
list_C = df['Year'].tolist()
while i<duplicity:
    QA = {}
    
    # parameters
    event_A = random.choice(list_A)
    list_B =[] 
    list_B = list_A.copy() 
    list_B.remove(event_A)
    event_B = random.choice(list_B)
    index_1 = list_A.index(event_A) 
    param_1 = list_C[index_1]


    parameters = [str(event_A),str(param_1),str(event_B)]
    
    #questions
    QA["question"] = replace_parameters(template_3,parameters)    
   
    # answer
    index_2 = list_A.index(event_B)
    if index_1>index_2: #event B happened before event A
        ans_t3 = randint(100,param_1-1)
        option_2 = randint(param_1+1,2020)
       
    else: #event B happened after event A
        ans_t3 = randint(param_1+1,2020)
        option_2 = randint(100,param_1-1)
          
    #options
    options_list = [ans_t3, option_2, param_1, 2020 + randint(1,1000)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t3
    print(QA["question"])
    
    json_dataset.append(QA)
    
    i+=1;

                                  Event  Year
0             first use of modern paper   105
1                  Unification of China   280
2     Rule of Roman Emperor Constantine   312
3                  Fall of Roman Empire   476
4           invention of printing press  1450
5                  Dicsovery of America  1492
6               Execution of Charles I   1649
7   Declaration of America Independence  1776
8              invention of vaccination  1796
9               discovery of anesthesia  1844
10                   Begin of Civil war  1861
11               invention of telephone  1876
12          invention of electric light  1879
13              invention of automobile  1885
14     invention of gas powered tractor  1892
15                   invention of radio  1895
16                invention of airplane  1903
17             Beginning of World War I  1914
18                   End of World War I  1918
19                invention of computer  1938
20             Outbreak of World W

In [775]:
#***********************************************************************************************************#
#  Template 4 : Based on the keywords (above, before, ahead etc) mentioned in Keyword_4 list               *#                      *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])
type_of_competition = ["race","competition","sport"]
keyword_4 = ["ahead of", "below", "before", "after", "beyond", "above", "later than","behind"]

template_4 = "In a $X, if $A scores $1 points and is $Z $B. Which of the following can be the score of $B?"

i=0
while i<duplicity:
    QA = {}
    
    # parameters
    competition = random.choice(type_of_competition)
    list_A = df['Names'].tolist()
    obj_A = random.choice(list_A)
    list_B =[] 
    list_B = list_A.copy() 
    list_B.remove(obj_A)
    obj_B = random.choice(list_B)
    param_1 = randint(1,100000)
    keyword = random.choice(keyword_4)

    parameters = [competition,obj_A,str(param_1),keyword,obj_B,obj_B]
    
    #questions
    QA["question"] = replace_parameters(template_4,parameters)    
   
    # answer
    if keyword == "ahead of" or keyword == "above" or keyword == "before" or keyword == "beyond": 
        ans_t4 = randint(0,param_1-1)
        option_2 = randint(param_1+1,1000000)
        option_3 = -1 * randint(param_1+1,1000000)
    else:
        ans_t4 = randint(param_1+1,1000000)
        option_2 = randint(0,param_1-1)
        option_3 = -1 * randint(0,param_1-1)
    
    #options
    options_list = [ans_t4, option_2, option_3]
    option_4 = random.choice(options_set)
    random.shuffle(options_list)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans
    print(QA["question"])
    print(QA["answer"])
    
    json_dataset.append(QA)
    
    i+=1;

In a sport, if Conor scores 4494 points and is before Harper. Which of the following can be the score of Harper?
36036
In a sport, if Chase scores 27254 points and is later than Jessie. Which of the following can be the score of Jessie?
36036
In a race, if Ariana scores 66766 points and is below Maya. Which of the following can be the score of Maya?
36036
In a competition, if Archer scores 78434 points and is below Jose. Which of the following can be the score of Jose?
36036
In a competition, if Abel scores 84689 points and is later than Jesse. Which of the following can be the score of Jesse?
36036
In a sport, if Lane scores 39873 points and is ahead of Tyson. Which of the following can be the score of Tyson?
36036
In a competition, if Caleb scores 51093 points and is above Ishaan. Which of the following can be the score of Ishaan?
36036
In a sport, if Boult scores 52232 points and is ahead of Amelia. Which of the following can be the score of Amelia?
36036
In a competition, if Miller

In [776]:
#***********************************************************************************************************#
#  Template 5 : Based on the the price of different items                                                  *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Price':str},sheet_name='template_5')  
df = pd.DataFrame(data,columns=['Price']) # Sorted as per descendig order of price
print(df)
template_5 = "Cost of $A is $1 dollars. How much could be the cost of $B?"

i=0
list_A = df['Price'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    obj_A = random.choice(list_A)
    list_B= []
    list_B = list_A.copy()
    list_B.remove(obj_A)    
    obj_B = random.choice(list_B)
    param_1 = randint(3,100000)
        
    parameters = [obj_A,str(param_1),obj_B]
    
    #questions
    QA["question"] = replace_parameters(template_5,parameters)    
    print(QA["question"])
    # answer
    index_1 = list_A.index(obj_A)
    index_2 = list_A.index(obj_B)
    if index_1 < index_2:   # Object A is expensive than Object B
        ans_t5 = randint(1,param_1-1)
        option_2 = randint(param_1+1,1000000)
        option_3 = -1 * randint(param_1+1,1000000)
    else:               # Object B is expensive than Object A 
        ans_t5 = randint(param_1+1,1000000)
        option_2 = randint(0,param_1-1)
        option_3 = -1 * randint(0,param_1-1)
    
    #options
    options_list = [ans_t5, option_2, option_3]
    option_4 = random.choice(options_set)
    random.shuffle(options_list)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t5
    
    json_dataset.append(QA)
    
    i+=1;

         Price
0   Space ship
1     Airplane
2         Ship
3   Helicopter
4        Villa
..         ...
93      Eraser
94    Matchbox
95      Thread
96      Needle
97       Candy

[98 rows x 1 columns]
Cost of Ship is 79895 dollars. How much could be the cost of Glue?
Cost of Casseroles is 23089 dollars. How much could be the cost of Eraser?
Cost of Stapler is 92942 dollars. How much could be the cost of Guitar?
Cost of Iphone 4 is 31555 dollars. How much could be the cost of Sports Bike?
Cost of Iphone 8 is 50829 dollars. How much could be the cost of Iphone 10 plus?
Cost of Diamond Ring is 29358 dollars. How much could be the cost of Smart Watch?
Cost of Sports Bike is 70823 dollars. How much could be the cost of Bicycle?
Cost of Tooth Paste is 72466 dollars. How much could be the cost of Jeans Pant?
Cost of Television is 42001 dollars. How much could be the cost of Villa?
Cost of Air Conditioner is 42292 dollars. How much could be the cost of Bike?
Cost of Trolley is 81124 dollars.

In [777]:
#***********************************************************************************************************#
#  Template 6 : Based on the speed of different modes of transport                                         *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Place':str,'Vehicle':str},sheet_name='template_6')  
df = pd.DataFrame(data,columns=['Place','Vehicle']) #Vehicles sorted in the ascending order of speed
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names':str},sheet_name='names')  
df_1 = pd.DataFrame(data_1,columns=['Names'])

time_list = ["hours","days","years","seconds","minutes","months"]
template_6 = "If $A takes $1 $B to reach $C by $X, then how long in $B it would take to travel by $Y?"

i=0
A_list = df_1['Names'].tolist()
C_list = df['Place'].tolist()
X_list = df['Vehicle'].tolist()

while i<duplicity:
    QA = {}
            
    # parameters 
    A = random.choice(A_list)
    param_1 = randint(4,10000)
    B = random.choice(time_list)    
    C = random.choice(C_list)
    X = random.choice(X_list)
    Y_list = []
    Y_list = X_list.copy()
    Y_list.remove(X)
    Y = random.choice(Y_list)
        
    parameters = [A,str(param_1),B,C,X,B,Y]
    
    #questions
    QA["question"] = replace_parameters(template_6,parameters)    
    print(QA["question"])
    # answer
    index_1 = X_list.index(X)
    index_2 = X_list.index(Y)
    if index_1>index_2: #X is faster than Y
        ans_t6 = randint(0,param_1-1)
        option_2 = randint(param_1+2,100001)
        options_list_no_ans = [0,param_1+1, -1 *option_2]
        option_3 = random.choice(options_list_no_ans )
    else: #Y is faster than X
        ans_t6 = randint(param_1+1,100001)
        option_2 = randint(0,param_1-2)
        options_list_no_ans = [0,param_1-1, -1 *option_2]
        option_3 = random.choice(options_list_no_ans )
    
    #options
    options_list = [ans_t6, option_2, option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t6
    
    json_dataset.append(QA)
    
    i+=1;

If Laila takes 2898 minutes to reach Louisiana by Trains, then how long in minutes it would take to travel by Steamers?
If Kathryn takes 7600 months to reach Charlotte by Small Yacht, then how long in months it would take to travel by Warships?
If Rocky takes 9760 years to reach Ocean by Super Sonic planes, then how long in years it would take to travel by Choppers?
If Jose takes 4190 minutes to reach McAllen by Airplanes, then how long in minutes it would take to travel by Trolley buses?
If Conor takes 6547 days to reach Beaumont by Taxis, then how long in days it would take to travel by Jet scooters?
If Carlos takes 8392 minutes to reach Salem by Jet Ski boats, then how long in minutes it would take to travel by Aircrafts?
If Kaylee takes 1687 days to reach Ocean by Space shuttles, then how long in days it would take to travel by Jet Ski boats?
If Rocky takes 2012 days to reach Norman by Gliders, then how long in days it would take to travel by Yacht?
If Mustafa takes 6883 seconds to

In [778]:
#***********************************************************************************************************#
#  Template 7 : Based on keywords mentioned in keywords_7_2  (leave, join , enter, exit etc..)              #
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Place':str},sheet_name='template_7')  
df = pd.DataFrame(data,columns=['Place'])
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Category':str},sheet_name='human_category')  
df_1 = pd.DataFrame(data_1,columns=['Category'])
print(df)
clock = ["am","pm"]
keywords_7_1 = ["Some","Few","A small number of","Less number of"]
keywords_7_2 = ["left","leave","departed","are withdrawn from","are sent out","exit","joined","entered","are merged"] 
split_index = 5 # in the above list elements in indexes 0 to split corresponds to subtraction, elements > split index corresponds to addition 
template_7 = "There are $1 $A in the $B at $t $x. $k $A $l the $B at $s $y. What could be the number of $A in the $B after $s $y?"

i=0
A_list = df_1['Category'].tolist()
B_list =df['Place'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    param_1 = randint(4,9999)
    A = random.choice(A_list)
    B = random.choice(B_list)
    x = random.choice(clock)
    y = random.choice(clock)
    if (x=="am" and y=="am") or (x=="pm" and y=="pm"):
        t = randint(1,10)
        s = randint(t,11)
    else:
        t = randint(1,12)
        s = randint(1,12)
    k = random.choice(keywords_7_1)
    l = random.choice(keywords_7_2)
        
    parameters = [str(param_1),A,B,str(t),x,k,A,l,B,str(s),y,A,B,str(s),y]
    
    #questions
    QA["question"] = replace_parameters(template_7,parameters)    
    print(QA["question"])
    # answer
    index_1 = keywords_7_2.index(l)
    if index_1 > split_index:    #  Add the no_of people entering
        ans_t7 = randint(param_1+1,10000)
        option_2 = randint(0,param_1-1)
    else:                       # Subtract the no_of people leaving
        ans_t7 = randint(0,param_1-1)
        option_2 = randint(param_1+1,10000)
    
    options_no_ans = [-1*option_2,0,-1*randint(1,10)*ans_t7]
    option_3 = random.choice(options_no_ans)
    
    #options
    options_list = [ans_t7, option_2,option_3,param_1]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t7
    print(QA["question"])
    json_dataset.append(QA)
    
    i+=1;

               Place
0             office
1          classroom
2             school
3         playground
4               camp
5         conference
6                lab
7           hospital
8              hotel
9               mall
10        board room
11           college
12           airport
13          bus stop
14   railway station
15            market
16           theater
17  shopping complex
18      meeting hall
19            temple
20               zoo
21           library
22             beach
23            museum
24              park
25     metro station
26       planetarium
27        restaurant
28            picnic
There are 500 actors in the bus stop at 10 pm. Few actors entered the bus stop at 11 pm. What could be the number of actors in the bus stop after 11 pm?
There are 500 actors in the bus stop at 10 pm. Few actors entered the bus stop at 11 pm. What could be the number of actors in the bus stop after 11 pm?
There are 194 children in the library at 3 pm. Some children are

In [779]:
#***********************************************************************************************************#
#  Template 8 : Based on the keywords less than, at most, at least etc..                                   *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item': str,'Container':str},sheet_name='template_8')  
df = pd.DataFrame(data,columns=['Item','Container'])
print(df)
keywords_8= ["a maximum of", "less than", "at most", "greater than","at least", "a minimum of","more than"]
split_index = 2
template_8= "There are $1 $A in $B. If $k $2 $A are lost, then how many $A will be remaining in the $B? "

A_list = df['Item'].tolist()
B_list =df['Container'].tolist()
i=0
while i<duplicity:
    QA = {}
            
    # parameters
    index = randint(0,len(A_list)-1)
    param_1 = randint(5,9999)
    A = A_list[index]
    B = B_list[index]
    k = random.choice(keywords_16)
    param_2 = randint(2,param_1-2) 
        
    parameters = [str(param_1),A,B,k,str(param_2),A,A,B]
    
    #questions
    QA["question"] = replace_parameters(template_8,parameters)    
    print(QA["question"])
    # answer
    index_1 = keywords_8.index(k)
    if k == "more than":    #remaining items <  initial no of items - items lost  
        ans_t8 = randint(0,param_1-param_2-1)
        option_3 = param_1-param_2
    elif k == "less than":  #remaining items >  initial no of items - items lost 
        ans_t8 = randint(param_1-param_2+1,param_1)
        option_3 = param_1-param_2
    elif index_1 > split_index:   #remaining items <=  initial no of items - items lost  
        ans_t8 = randint(0,param_1-param_2)
        option_3 = randint(param_1-param_2+1,param_1)
    else:                 #remaining items >=  initial no of items - items lost       
        ans_t8 = randint(param_1-param_2,param_1)
        option_3 = randint(0,param_1-param_2-1)

    #options
    options_list = [ans_t8,param_1,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t8
    
    json_dataset.append(QA)
    
    i+=1;

             Item Container
0          apple     fridge
1         bananas    basket
2   sugar packets       jar
3          houses    colony
4         oranges    basket
5      chocolates       jar
6          candys    basket
7        tea bags       jar
8         oranges    fridge
9          grapes    fridge
10        bananas    fridge
11           pens      tray
12        pencils      tray
13           pens       bag
14     chocolates    fridge
15      dryfruits    fridge
16   luggage bags       bus
17   luggage bags     train
18   luggage bags  airplane
19   luggage bags      ship
20   luggage bags      taxi
21   luggage bags       car
22          coins       box
23         apples    basket
24         grapes    basket
25       tomatoes    basket
26       potatoes    basket
27        brinjal    basket
28           kiwi    basket
29        mangoes    basket
30       cucumber    basket
31       tomatoes    fridge
32       potatoes    fridge
33        brinjal    fridge
34           kiwi   

In [780]:
#***********************************************************************************************************#
#  Template 9 :  Larger vs Smaller objects                                                                 *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item':str,'Use':str},sheet_name='template_9')  
df = pd.DataFrame(data,columns=['Item','Use'])
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names':str},sheet_name='names')  
df_1 = pd.DataFrame(data_1,columns=['Names'])

template_9 = "If $A needs $1 $B to $C, then how many $D will be needed to $C?"

i=0
def find(lst,a):
    result = []
    j=0;
    for x in lst:
        if x == a:
            result.append(j)
        j = j+1
    return result

names =df_1['Names'].tolist()
B_list =df['Item'].tolist() #items
C_list = df['Use'].tolist() # use case of item B
while i<duplicity:
    QA = {}           
    # parameters
    A = random.choice(names)
    param_1 = randint(2,10000)
    index = randint(0,len(B_list)-1)
    B = B_list[index]
    C = C_list[index] 
    D_list = []
    D_list = find(C_list,C)    # list of item indexes with same use case
    D_list.remove(index)
    index_choice = random.choice(D_list)
    D = B_list[index_choice]
        
    parameters = [A,str(param_1),B,C,D,C]
    
    #questions
    QA["question"] = replace_parameters(template_9,parameters)    
    print(QA["question"])
    # answer
    index_1 = index
    index_2 = index_choice
    if index_2>index_1: # Second item (D) is bigger than first item(B), so need less number of D items than B
        ans_t9 = randint(1,param_1-1)
        option_3 = randint(param_1+1,100000)
    else:
        ans_t9 = randint(param_1+1,100000)
        option_3 = randint(1,param_1-1)

    #options
    options_list = [ans_t9,param_1,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t9
    
    json_dataset.append(QA)
    
    i+=1;

If Lane needs 5742 glass to fill the tank, then how many bucket will be needed to fill the tank?
If Ivan needs 6386 bucket to fill the tank, then how many mug will be needed to fill the tank?
If Nathan needs 9964 tomatoes to fill the container, then how many apples will be needed to fill the container?
If Misael needs 7067 strawberries to fill the container, then how many tomatoes will be needed to fill the container?
If Caden needs 9729 oranges to fill the container, then how many potatoes will be needed to fill the container?
If Hector needs 6806 Grapes to fill the container, then how many tomatoes will be needed to fill the container?
If Carolina needs 2992 jar to fill the tank, then how many mug will be needed to fill the tank?
If Carolina needs 9874 mangoes to fill the container, then how many Cherries will be needed to fill the container?
If Alivia needs 6537 Cherries to fill the container, then how many tomatoes will be needed to fill the container?
If Asher needs 8439 mug to fi

In [781]:
#***********************************************************************************************************#
#  Template 10 : Unequal division (Change in Length)                                                       *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item':str},sheet_name='template_10')  
df = pd.DataFrame(data,columns=['Item'])
print(df)


template_10 = "A $A is divided into $1 parts. Length of one of the parts is $2 $k. What could be the length of $A before division in k? "

i=0
A_list =df['Item'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    A = random.choice(A_list)
    param_1 = randint(2,100)
    param_2 = randint(3,1000)
    k = random.choice(measurement_height)
    parameters = [A,str(param_1),str(param_2),k,A,k]
    
    #questions
    QA["question"] = replace_parameters(template_10,parameters)    
    print(QA["question"])
    # answer
    ans_t10 = randint(param_2+1,100000)

    #options
    options_list = [ans_t10,randint(0,param_2-1),param_2,int(param_2/param_1)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t10
    
    json_dataset.append(QA)
    
    i+=1;

          Item
0        stick
1        trunk
2         time
3       pencil
4     triangle
5       circle
6       square
7    rectangle
8     pentagon
9      hexagon
10      sphere
11       apple
12      orange
13  watermelon
14     pumpkin
15        cube
16       cloth
17        cake
18       paper
19        rope
20        wire
21   cardboard
22  metal foil
23      thread
24     octagon
25     rhombus
26     ellipse
27        oval
28     nonagon
29     polygon
30        kite
31       pizza
32     napkins
33   pineapple
34        pipe
35      string
36      ribbon
37  jack fruit
38     blanket
39       towel
A rhombus is divided into 51 parts. Length of one of the parts is 754 feet. What could be the length of rhombus before division in k? 
A pumpkin is divided into 93 parts. Length of one of the parts is 3 inches. What could be the length of pumpkin before division in k? 
A polygon is divided into 69 parts. Length of one of the parts is 131 inches. What could be the length of polygon b

In [782]:
#***********************************************************************************************************#
#  Template 11 : Based on collective nouns                                                                 *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Collective_noun':str,'Persons':str,'Action':str},sheet_name='template_11')  
df = pd.DataFrame(data,columns=['Collective_noun','Persons','Action'])
print(df)
#Collective noun consists of atleast two items/people of the same type. So if x number of people/items join, then the total
# number of people could be atleast x+2

template_11 = "In a $A of $B, if $1 $B $C then how many $B could be the present in the $A? "

i=0
A_list =df['Collective_noun'].tolist()
B_list =df['Persons'].tolist()
C_list = df['Action'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters 
    index = randint(0,len(A_list)-1)
    A = A_list[index]  
    B = B_list[index] 
    C = C_list[index] 
    param_1 = randint(2,100000)
    
    parameters = [A,B,str(param_1),B,C,B,A]
    
    #questions
    QA["question"] = replace_parameters(template_11,parameters)    
    print(QA["question"])
    # answer
    ans_t11 = randint(param_1+2,100000+param_1)

    #options
    options_list = [ans_t11,param_1,param_1+1,randint(0,param_1)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t11
    
    json_dataset.append(QA)
    
    i+=1;

  Collective_noun   Persons Action
0            army  soldiers   join
1            team   players  enter
2            band   singers   join
3           bunch    people  merge
In a bunch of people, if 6000 people merge then how many people could be the present in the bunch? 
In a band of singers, if 88805 singers join then how many singers could be the present in the band? 
In a bunch of people, if 89747 people merge then how many people could be the present in the bunch? 
In a band of singers, if 28938 singers join then how many singers could be the present in the band? 
In a army of soldiers, if 43488 soldiers join then how many soldiers could be the present in the army? 
In a army of soldiers, if 93104 soldiers join then how many soldiers could be the present in the army? 
In a army of soldiers, if 45354 soldiers join then how many soldiers could be the present in the army? 
In a army of soldiers, if 41577 soldiers join then how many soldiers could be the present in the army? 
In a b

In [783]:
#***********************************************************************************************************#
#  Template 12 : Based on nouns describing capacity                                                        *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Capacity':str,'Items':str},sheet_name='template_12')  
df = pd.DataFrame(data,columns=['Capacity','Items']) # Nouns sorted by capacity
print(df)


template_12 = "If the weight of $A of $C is $1 $k. What could be the weight of $B of $C in $k? "

i=0
while i<duplicity:
    QA = {}
            
    # parameters
    A_list =df['Capacity'].tolist()
    A = random.choice(A_list)
    B_list =[] 
    B_list = A_list.copy()
    B_list.remove(A)
    B = random.choice(B_list)
    k = random.choice(measurement_weight)
    param_1 = randint(2,100000)
    C_list = df['Items'].tolist()
    C = random.choice(C_list)
    
    parameters = [A,C,str(param_1),k,B,C,k]
    
    #questions
    QA["question"] = replace_parameters(template_12,parameters)    
    print(QA["question"] )
    # answer
    index_1 = A_list.index(A)
    index_2 = A_list.index(B)
    if index_1 > index_2: # B is smaller than A
        ans_t12 = randint(1,param_1-1)
        option_2 = randint(param_1+1,100000+param_1)
    else: # B is larger than A
        ans_t12 = randint(param_1+1,100000+param_1)
        option_2 = randint(1,param_1-1)
    

    #options
    options_list = [ans_t12,option_2,param_1]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t12
    
    json_dataset.append(QA)
    
    i+=1;

       Capacity          Items
0       handful         spoons
1        armful        oranges
2        cupful         apples
3      houseful         grapes
4   teaspoonful        bananas
5       tankful  sugar packets
6     bucketful   bell peppers
7     basketful          pears
8    barrelsful     chocolates
9    platterful         candys
10   kitchenful       tea bags
11    pocketful     pineapples
12     spoonful       apricots
13    bottleful        papayas
14     glassful           pens
15     plateful        pencils
16     mouthful       avocados
17   tumblerful     chocolates
18     shelfful      Dryfruits
19     tableful         melons
20     scoopful        peaches
21     trunkful          plums
22    teacupful   pomegranates
23      roomful       coconuts
24     sacksful      eggplants
25      pipeful   french beans
26      rackful       chillies
27      cageful         onions
28      dishful       tomatoes
29      bowlful       potatoes
30      palmful        carrots
31      

In [784]:
#***********************************************************************************************************#
#  Template 13 : With and without                                                                          *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Items':str,'With':str,'Without':str,"Affects":str},sheet_name='template_13')  
df = pd.DataFrame(data,columns=['Items','With','Without','Affects'])
print(df)


template_13 = "If the $l of $A $B is $1 $k. How much is the $l of $A $C? "

i=0
A_list =df['Items'].tolist()
X_list = df['With'].tolist()
Y_list = df['Without'].tolist()
l_list = df['Affects'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    index = randint(0,len(A_list)-1)
    A = A_list[index]
    l = l_list[index]
    choice = randint(0,1)
    if choice==0:
        B = X_list[index]
        C = Y_list[index]
    else:
        B = Y_list[index]
        C = X_list[index]
    k = random.choice(measurement_weight)
    param_1 = randint(3,100000)
    
    parameters = [l,A,B,str(param_1),k,l,A,C]
    
    #questions
    QA["question"] = replace_parameters(template_13,parameters)    
    print(QA["question"] )
    # answer
    if choice == 0: #  C < B
        ans_t13 = randint(1, param_1-1)
        option_2 = randint(param_1+1,1000000) 
    else: #weight of C> weight of B
        ans_t13 = randint(param_1+1,1000000)
        option_2 = randint(1, param_1-1)

    #options
    options_list = [ans_t13,option_2,param_1,0]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t13
    
    json_dataset.append(QA)
    
    i+=1;

           Items                With            Without       Affects
0          Hands         with gloves     without gloves        weight
1        carpet          when folded      when unfolded  surface area
2         apples        inside a box        with no box        weight
3         ballon     filled with air        with no air        weight
4        bananas   warapped in cover          uncovered        weight
5           legs          with socks      without socks        weight
6        blanket         when folded      when unfolded  surface area
7          shirt         when folded      when unfolded  surface area
8        oranges     inside a basket     with no basket        weight
9       tomatoes  inside a container  without container        weight
10      potatoes        inside a box        with no box        weight
11        grapes    wrapped in cover          uncovered        weight
12         cloth         when folded      when unfolded  surface area
13       mangoes    

If the weight of mangoes inside a basket is 39276 kgs. How much is the weight of mangoes with no basket? 
If the weight of pineapples inside a container is 65501 kgs. How much is the weight of pineapples without container? 
If the weight of tomatoes inside a container is 54234 grams. How much is the weight of tomatoes without container? 
If the weight of bucket filled with water is 51678 milligrams. How much is the weight of bucket without water? 
If the weight of ballon filled with air is 33841 pounds. How much is the weight of ballon with no air? 
If the weight of bucket filled with water is 76772 milligrams. How much is the weight of bucket without water? 
If the weight of water melons without container is 54365 pounds. How much is the weight of water melons inside a container? 
If the weight of apples with no box is 94805 grams. How much is the weight of apples inside a box? 
If the weight of candys packed in cover is 63803 pounds. How much is the weight of candys uncovered? 
If th

In [785]:
#***********************************************************************************************************#
#  Template 14 : Calculate the earnings                                                                     *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])   
names = df['Names'].tolist()
template_14 = "$1 has $2 $3. $4 gives the $5 on rent, each for $6 dollars per day. On $7 he does not take any money for them. In a week, how much does $1 earn from $8 $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    no_of_items = randint(2,20)
    item = random.choice(items_for_rent_list)
    price = no_of_items*randint(1,100)
    day = random.choice(days_in_a_week)
    no_of_items_in_qn = randint(1,no_of_items) 
    parameters = [name, str(no_of_items),item,name,item,str(price),day,name,str(no_of_items_in_qn),item]
    
    #questions
    QA["question"] = replace_parameters(template_14,parameters)    
    print(QA["question"])
  
    # answers
    earnings = int(price/no_of_items)*6*no_of_items_in_qn
    
    #options
    options_list = [earnings,randint(1,100),randint(0,price),randint(price,earnings*5)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = earnings
    
    json_dataset.append(QA)
    i+=1;
    

Rowan has 15 cars. Rowan gives the cars on rent, each for 690 dollars per day. On Friday he does not take any money for them. In a week, how much does Rowan earn from 9 cars?
Rocky has 15 bikes. Rocky gives the bikes on rent, each for 300 dollars per day. On Monday he does not take any money for them. In a week, how much does Rocky earn from 14 bikes?
Zahir has 3 buses. Zahir gives the buses on rent, each for 99 dollars per day. On Monday he does not take any money for them. In a week, how much does Zahir earn from 1 buses?
Ronan has 11 cars. Ronan gives the cars on rent, each for 1078 dollars per day. On Sunday he does not take any money for them. In a week, how much does Ronan earn from 10 cars?
Rowan has 4 buses. Rowan gives the buses on rent, each for 304 dollars per day. On Thursday he does not take any money for them. In a week, how much does Rowan earn from 4 buses?
Rocky has 8 cars. Rocky gives the cars on rent, each for 440 dollars per day. On Wednesday he does not take any mo

In [786]:
template_15 = "$1 has small $2. $3 uses $4 of them to fill one large $5 of capacity $6 $7. How much is the capacity of one small $8 in $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    container = random.choice(containers_list)
    no_of_small_containers = randint(2,20)
    capacity_of_large_container = no_of_small_containers*randint(1,30)
    measure = random.choice(measurement_liquid)
    parameters = [name, container,name, str(no_of_small_containers), container,str(capacity_of_large_container),measure,container,measure]
  
    #questions
    QA["question"] = replace_parameters(template_15,parameters)    
    print(QA["question"])
    # answer
    capacity_of_small_container= int(capacity_of_large_container/no_of_small_containers)
    
    #options
    options_list = [capacity_of_small_container,capacity_of_large_container,randint(0,capacity_of_small_container-1),randint(capacity_of_small_container+1,capacity_of_large_container-1)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = capacity_of_small_container
    
    json_dataset.append(QA)
    
    i+=1;

Regina has small jars. Regina uses 12 of them to fill one large jars of capacity 348 ml. How much is the capacity of one small jars in ml?
Veronica has small tanks. Veronica uses 10 of them to fill one large tanks of capacity 270 ounces. How much is the capacity of one small tanks in ounces?
Nickolas has small bowls. Nickolas uses 17 of them to fill one large bowls of capacity 238 L. How much is the capacity of one small bowls in L?
Patrick has small bowls. Patrick uses 6 of them to fill one large bowls of capacity 66 ounces. How much is the capacity of one small bowls in ounces?
Leon has small bowls. Leon uses 14 of them to fill one large bowls of capacity 364 gallons. How much is the capacity of one small bowls in gallons?
Sunny has small tanks. Sunny uses 5 of them to fill one large tanks of capacity 140 litres. How much is the capacity of one small tanks in litres?
Leon has small bowls. Leon uses 13 of them to fill one large bowls of capacity 221 litres. How much is the capacity of

In [787]:
template_16 = "$1 bought $2 $3 of $4 from the market for $5 dollars. How much more does $6 need to pay to buy a total of $7 $8 of $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    purchased_items = randint(2,50)
    measure = random.choice(measurement_weight)
    veggie = random.choice(vegetables)
    price = randint(2,50) * purchased_items
    no_of_items = randint(purchased_items+1,100) 
    parameters = [name, str(purchased_items),measure,veggie, str(price),name,str(no_of_items), measure, veggie]
  
    #questions
    QA["question"] = replace_parameters(template_16,parameters)    
    print(QA["question"])
    # answer
    price_ans = int(((price/purchased_items)*no_of_items) - price)
    
    #options
    options_list = [int(price_ans),price,randint(0,price-1),randint(0,price_ans*6)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = price_ans
    
    json_dataset.append(QA)
    
    i+=1;

Jaden bought 5 gm of mushrooms from the market for 225 dollars. How much more does Jaden need to pay to buy a total of 73 gm of mushrooms?
Alan bought 13 gm of mushrooms from the market for 559 dollars. How much more does Alan need to pay to buy a total of 43 gm of mushrooms?
Nathan bought 25 kgs of capsicum from the market for 1075 dollars. How much more does Nathan need to pay to buy a total of 31 kgs of capsicum?
Anson bought 13 kgs of tomatoes from the market for 598 dollars. How much more does Anson need to pay to buy a total of 25 kgs of tomatoes?
Jude bought 39 mg of mushrooms from the market for 429 dollars. How much more does Jude need to pay to buy a total of 87 mg of mushrooms?
Bryan bought 45 grams of onions from the market for 2115 dollars. How much more does Bryan need to pay to buy a total of 79 grams of onions?
Boult bought 16 grams of onions from the market for 592 dollars. How much more does Boult need to pay to buy a total of 68 grams of onions?
Cole bought 10 grams 

In [788]:

template_17 = "If a person who owns a $1 earns $2 dollars in a day, how much will he earn in $3 $4?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    owns = random.choice(store_type)
    earns = randint(2,1000)
    duration_time = randint(2,1000) 
    duration = random.choice(duration_list)
    parameters = [owns, str(earns), str(duration_time),duration]
    
    #questions
    QA["question"] = replace_parameters(template_17,parameters)    
    print(QA["question"])
    # answer
    if duration == "years":
        t6_ans = earns*30*365*duration_time
    elif duration == "months":
        t6_ans = earns*30*duration_time
    elif duration == "weeks":
        t6_ans = earns*7*duration_time
    else:
        t6_ans = earns*duration_time
        
    
    
    #options
    options_list = [t6_ans,earns,randint(0,earns*5),randint(0,price_ans*6)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t6_ans)
    
    json_dataset.append(QA)
    
    i+=1;


If a person who owns a field earns 126 dollars in a day, how much will he earn in 966 weeks?
If a person who owns a school earns 844 dollars in a day, how much will he earn in 856 weeks?
If a person who owns a grocery store earns 322 dollars in a day, how much will he earn in 346 days?
If a person who owns a school earns 425 dollars in a day, how much will he earn in 893 days?
If a person who owns a school earns 274 dollars in a day, how much will he earn in 787 years?
If a person who owns a super market earns 384 dollars in a day, how much will he earn in 121 weeks?
If a person who owns a field earns 382 dollars in a day, how much will he earn in 814 days?
If a person who owns a grocery store earns 418 dollars in a day, how much will he earn in 661 weeks?
If a person who owns a salon earns 506 dollars in a day, how much will he earn in 305 days?
If a person who owns a super market earns 738 dollars in a day, how much will he earn in 667 weeks?
If a person who owns a field earns 453 do

In [789]:
template_18 = "Two $1 plan to $2 along a $3 $4 trail, starting at opposite ends of the trail at the same time. If $P's rate is $5 % $6 than $Q's, how many $7 will $X have walked when they pass each other?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    type_of_people = random.choice(human_category)
    activity = random.choice(activity_list)
    length_of_trail = randint(4,1000)
    measure = random.choice(measurement_distance)
    percent = randint(2,50)
    alpha = 'A'
    index = randint(0,24)
    alpha_1 = chr(ord(alpha) + index)
    alpha_2 = chr(ord(alpha) + (index+1))
    alpha_list = [alpha_1,alpha_2]
    question_alpha = random.choice(alpha_list)
    fast_or_slow = ["faster","slower"]
    qn_keyword = random.choice(fast_or_slow)
    
    parameters = [type_of_people, activity, str(length_of_trail),measure,str(alpha_1),str(percent),qn_keyword,str(alpha_2),measure,str(question_alpha)]
    
    #questions
    QA["question"] = replace_parameters(template_18,parameters)    
    print(QA["question"])
    # answer
    if qn_keyword == "faster":
        x = (1+(percent/100))*(length_of_trail/(1+(1+(percent/100))))
        if question_alpha == alpha_1:
            t7_ans = x
        else:
            t7_ans = length_of_trail - x
    else:
        x = (1-(percent/100))*(length_of_trail/(1+(1-(percent/100))))
        if question_alpha == alpha_1:
            t7_ans = x
        else:
            t7_ans = length_of_trail - x
    
    
    #options
    options_list = [int(t7_ans),int(length_of_trail/2),randint(0,length_of_trail-3),percent]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t7_ans)
    
    json_dataset.append(QA)
    
    i+=1;


Two children plan to jog along a 427 miles trail, starting at opposite ends of the trail at the same time. If D's rate is 19 % slower than E's, how many miles will E have walked when they pass each other?
Two students plan to jog along a 22 km trail, starting at opposite ends of the trail at the same time. If O's rate is 28 % slower than P's, how many km will P have walked when they pass each other?
Two travellers plan to brisk walk along a 218 miles trail, starting at opposite ends of the trail at the same time. If J's rate is 36 % faster than K's, how many miles will K have walked when they pass each other?
Two persons plan to race along a 364 mts trail, starting at opposite ends of the trail at the same time. If D's rate is 38 % faster than E's, how many mts will D have walked when they pass each other?
Two friends plan to jog along a 777 mm trail, starting at opposite ends of the trail at the same time. If S's rate is 34 % faster than T's, how many mm will S have walked when they p

In [790]:
Lenght = ["km","m","miles"]
Time = ["sec","hr","min"]
#Since Speed = Length/Time
def convert_length(val,len1,len2):
    if len1 == len2:
        return val
    else:
        if len1=="km" and len2 == "m":
            return val*1000
        elif len1=="m" and len2 == "km":
            return val/1000
        elif len1=="km" and len2 == "miles":
            return val*0.62 
        elif len1=="miles" and len2 == "km":
            return val/0.62
        elif len1== "m" and len2=="miles":
            return val*0.000621371
        else:
            return val/0.000621371
        
def convert_time(val,time1,time2):
    if time1 == time2:
        return val
    else:
        if time1=="min" and time2 == "sec":
            return val/60
        elif time2=="min" and time1 == "sec":
            return val*60
        elif time2=="hr" and time1 == "min":
            return val/60
        elif time1=="hr" and time2 == "min":
            return val*60 
        elif time1=="hr" and time2 == "sec":
            return val/(60*60)
        else:
            return val*(60*60)
                

template_19 = "The speed at which a man can row a boat in still water is $1 $Xp$Y. If he rows $Y, where the speed of current is $2 $Xp$Y, what time in $W will he take to cover $3 $Z?"
i=0
while i<15:
    QA = {}
    
    # parameters
    speed_of_man = randint(4,1000)
    measure_length = random.choice(Lenght)
    measure_time = random.choice(Time)
    directions=["upstream","downstream"]
    dir = random.choice(directions)
    speed_of_curr = randint(2,speed_of_man-1)
    dist = randint(5,100)
    dist_length = random.choice(Lenght)
    dist_time = random.choice(Time)
    
    parameters = [str(speed_of_man), measure_length,measure_time, dir, str(speed_of_curr), measure_length,measure_time,dist_time,str(dist),dist_length]
    
    #questions
    QA["question"] = replace_parameters(template_19,parameters)    
    print(QA["question"])
    # answer
    if dir == "downstream":
        x = speed_of_man+speed_of_curr
        L = convert_length(x,measure_length,dist_length)  
        T = convert_time(L,measure_time,dist_time)       
           
    else:
        x = speed_of_man-speed_of_curr
        L = convert_length(x,measure_length, dist_length)  
        T = convert_time(L,measure_time,dist_time)
        
    t8_ans = int(dist/T)   
    #options
    options_list = [int(t8_ans),speed_of_man-speed_of_curr,speed_of_man+speed_of_curr,speed_of_curr]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t8_ans)
    
    if t8_ans != 0:
        json_dataset.append(QA)
    
    i+=1;

The speed at which a man can row a boat in still water is 662 milespsec. If he rows upstream, where the speed of current is 646 milespsec, what time in min will he take to cover 68 m?
The speed at which a man can row a boat in still water is 582 milespsec. If he rows downstream, where the speed of current is 121 milespsec, what time in hr will he take to cover 60 km?
The speed at which a man can row a boat in still water is 112 kmpmin. If he rows downstream, where the speed of current is 91 kmpmin, what time in min will he take to cover 30 km?
The speed at which a man can row a boat in still water is 52 kmpsec. If he rows downstream, where the speed of current is 6 kmpsec, what time in min will he take to cover 77 m?
The speed at which a man can row a boat in still water is 68 mphr. If he rows downstream, where the speed of current is 12 mphr, what time in hr will he take to cover 97 m?
The speed at which a man can row a boat in still water is 868 kmpmin. If he rows downstream, where t

In [791]:
template_20 = "In the coordinate plane, points ($1, $x) and ($y, $2) are on line $a. If line $a passes through the origin and has slope $3, then what are the values of $x and $y respectively?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    alpha = 'a'
    index = randint(0,24)
    alpha_1 = chr(ord(alpha) + index)
    alpha_2 = chr(ord(alpha) + (index+1))
    big_aplha = 'A'
    index_line = randint(0,24)
    aplha_3 = chr(ord(alpha) + index_line)
    x1 = randint(0,10)
    y2 = randint(0,20)
    slope = random.random()
    
    parameters = [str(x1),str(alpha_1),str(alpha_2),str(y2),str(aplha_3),str(aplha_3),str(slope),str(alpha_1),str(alpha_2)]
    
    #questions
    QA["question"] = replace_parameters(template_20,parameters)    
    print(QA["question"])
    # answer
    alpha1_x = randint(1,20)
    alpha2_y = int(alpha1_x*slope)
    
    
    #options
    options_list = [str(alpha1_x) +"and"+str(alpha2_y),str(alpha1_x+2) +"and"+str(alpha2_y-1),str(0)+"and"+str(alpha2_y-8),str(int(slope))+"and"+str(0)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = str(alpha1_x)+"and"+str(alpha2_y)
    
    json_dataset.append(QA)
    
    i+=1;

In the coordinate plane, points (10, l) and (m, 10) are on line c. If line c passes through the origin and has slope 0.5018896611822444, then what are the values of l and m respectively?
In the coordinate plane, points (6, v) and (w, 2) are on line j. If line j passes through the origin and has slope 0.48514203630472574, then what are the values of v and w respectively?
In the coordinate plane, points (9, w) and (x, 18) are on line l. If line l passes through the origin and has slope 0.5524906802884499, then what are the values of w and x respectively?
In the coordinate plane, points (5, m) and (n, 8) are on line h. If line h passes through the origin and has slope 0.3167659638648672, then what are the values of m and n respectively?
In the coordinate plane, points (7, m) and (n, 4) are on line m. If line m passes through the origin and has slope 0.5501923978831427, then what are the values of m and n respectively?
In the coordinate plane, points (2, r) and (s, 7) are on line o. If lin

In [792]:
food_expenses = ["food","lunch","dinner","meal","breakfast","coffee","buffet"]
template_21 = "If $1 had $2 at $3 and he gave $4% tip, how much did he spend?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    expense_item = random.choice(food_expenses)
    price = randint(10,500)
    tip_percent = randint(1,50)
    
    parameters = [name,expense_item,str(price),str(tip_percent)]
    
    #questions
    QA["question"] = replace_parameters(template_21,parameters)    
    print(QA["question"])
    # answer
    money_spent = int(price+price*(tip_percent/100))  
    
    
    #options
    options_list = [money_spent, randint(0,money_spent-1),price,price*randint(0,10)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = money_spent
    
    json_dataset.append(QA)
    
    i+=1;

If Bretlee had breakfast at 495 and he gave 3% tip, how much did he spend?
If Marcus had food at 411 and he gave 31% tip, how much did he spend?
If William had meal at 74 and he gave 1% tip, how much did he spend?
If Esther had coffee at 293 and he gave 8% tip, how much did he spend?
If Tyler had food at 185 and he gave 15% tip, how much did he spend?
If Travis had food at 328 and he gave 13% tip, how much did he spend?
If Henry had meal at 89 and he gave 23% tip, how much did he spend?
If Peter had buffet at 138 and he gave 39% tip, how much did he spend?
If Micah had breakfast at 48 and he gave 1% tip, how much did he spend?
If Kevin had breakfast at 164 and he gave 29% tip, how much did he spend?
If Robin had dinner at 384 and he gave 11% tip, how much did he spend?
If Jude had buffet at 182 and he gave 25% tip, how much did he spend?
If Kaylee had breakfast at 266 and he gave 48% tip, how much did he spend?
If Marcus had meal at 198 and he gave 7% tip, how much did he spend?
If Pla

In [793]:
#***********************************************************************************************************#
#  Template 22 : Based on keywords gets, removes, takes away etc                                          *#
#***********************************************************************************************************#
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item': str,'Container':str},sheet_name='template_8')  
df_1 = pd.DataFrame(data_1,columns=['Item','Container'])
print(df_1)
# based on keywords get/put
Keywords_22 = ["gets","removes","takes away","puts","places in","adds"]
split_index = 2
template_22 = "In a $A if $X $k $1 $B and $Y $l $2 $B, then how many $B could be present in $A? "

i=0
while i<duplicity:
    QA = {}
            
    # parameters
    A_list =df_1['Container'].tolist()
    index = randint(0,len(A_list)-1)
    A = A_list[index]
    B_list =df_1['Item'].tolist()  
    B = B_list[index]
    X = random.choice(names)
    Y = random.choice(names)
    k = random.choice(Keywords_22)
    l=random.choice(Keywords_22)
    param_1 = randint(2,100000)
    param_2 = randint(2,100000)
    
    parameters = [A,X,k,str(param_1),B,Y,l,str(param_2),B,B,A]
    
    #questions
    QA["question"] = replace_parameters(template_22,parameters)    
    print(QA["question"])
    # answer
    index_1 = Keywords_11.index(k)
    index_2 = Keywords_11.index(l)
    if index_1 <= split_index and index_2 <= split_index:
        ans_t22 = randint(param_1+param_2,1000000)
        option_2 = param_1+param_2-1
        option_3 = (param_1+param_2) - randint(2,1000)
    elif index_1 <= split_index and index_2 > split_index:
        ans_t22 = randint(param_2,1000000)
        option_2 = param_2-1
        option_3 = param_2 - randint(0,1000)
    elif index_1 > split_index and index_2 <= split_index:
        ans_t22 = randint(param_1,1000000)
        option_2 = param_1-1
        option_3 = param_1 - randint(0,1000)
    else:
        ans_t22 = randint(param_1+param_2,1000000)
        option_2 = param_1+param_2-1
        option_3 = (param_1+param_2) - randint(2,1000)

    #options
    options_list = [ans_t22,option_2,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t22
    
    json_dataset.append(QA)
    
    i+=1;

             Item Container
0          apple     fridge
1         bananas    basket
2   sugar packets       jar
3          houses    colony
4         oranges    basket
5      chocolates       jar
6          candys    basket
7        tea bags       jar
8         oranges    fridge
9          grapes    fridge
10        bananas    fridge
11           pens      tray
12        pencils      tray
13           pens       bag
14     chocolates    fridge
15      dryfruits    fridge
16   luggage bags       bus
17   luggage bags     train
18   luggage bags  airplane
19   luggage bags      ship
20   luggage bags      taxi
21   luggage bags       car
22          coins       box
23         apples    basket
24         grapes    basket
25       tomatoes    basket
26       potatoes    basket
27        brinjal    basket
28           kiwi    basket
29        mangoes    basket
30       cucumber    basket
31       tomatoes    fridge
32       potatoes    fridge
33        brinjal    fridge
34           kiwi   

In [794]:
import json
with open('dataset.json','w',encoding = 'utf-8') as f:
    json.dump(json_dataset,f,ensure_ascii=False,indent=4)